In [8]:
import datetime
import os
import matplotlib.pyplot as plt

import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm.autonotebook import tqdm

from efficientdet.utils import BBoxTransform, ClipBoxes
from utils.utils import preprocess, invert_affine, postprocess
import glob
import pandas as pd
import cv2
from backbone import EfficientDetBackbone
from efficientdet.dataset import CocoDataset, Resizer, Normalizer, Augmenter, collater


from shapely.geometry import Polygon
from pyproj import Transformer
import ast
import json
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [9]:
def geo_coords(box, x_translation, y_translation, geo_data, center=False):
  """
  Gets coordinates in epsg:4326 format (google maps) of bounding box
  If center, return only coordinate of center of bounding box
  Params:
    Box: [xmin, ymin, xmax, ymax]
  """
  transformer = Transformer.from_crs(geo_data['crs'], 'epsg:4326')
  if center:
    center_x, center_y = (box[2] + box[0])//2 + x_translation, (box[3] + box[1])//2 + y_translation
    new_coords = np.array(geo_data['transform']).reshape((3,3)) @ np.array( [center_x, center_y, 1])

    return transformer.transform(new_coords[0], new_coords[1])

  else:
    new_box = np.array([[x_translation + box[0], y_translation + box[1], 1],
            [x_translation + box[2], y_translation + box[3], 1]])
    new_coords = np.array(geo_data['transform']).reshape((3,3)) @ new_box.T

    return transformer.transform(new_coords[0], new_coords[1])

def get_list_polygon(xs, ys):
    xs, ys = sorted(xs), sorted(ys)
    return [[ys[0], xs[0]], [ys[1], xs[0]], [ys[1], xs[1]], [ys[0], xs[1]]]


input_sizes = [512, 640, 768, 896, 1024, 1280, 1280, 1536, 1536]

class EvaluationSet(object):
    def __init__(self, img_path, set_name='val', compound_coeff=1,
                 mean=[0.3470, 0.3371, 0.3029], std=[0.2138, 0.2029, 0.1984]):
        
        self.imgs = sorted(glob.glob(f'{img_path}/images/{set_name}/*'))
        self.df = pd.read_csv(f'{img_path}/{set_name}.csv')
        self.mean = mean
        self.std = std
        self.compound_coeff = compound_coeff
        self.max_size = input_sizes[compound_coeff]
        
    def __getitem__(self, idx):
        return preprocess(self.imgs[idx], max_size=self.max_size, mean=self.mean, std=self.std)
    
    def __len__(self):
        return len(self.imgs)

In [10]:
import tqdm
        
def compute_coordinates(dataset, model, device, nms_threshold=0.45, threshold=0.2, set_name='val'):
    results = []
    json_results = []
    regressBoxes = BBoxTransform()
    clipBoxes = ClipBoxes()
    
    for idx, (ori_imgs, framed_imgs, framed_metas) in enumerate(tqdm.tqdm(dataset)):

        x = torch.from_numpy(framed_imgs[0])

        x = x.to(device)

        x = x.unsqueeze(0).permute(0, 3, 1, 2)
        features, regression, classification, anchors = model(x)

        preds = postprocess(x,
                            anchors, regression, classification,
                            regressBoxes, clipBoxes,
                            threshold, nms_threshold)
        
        if not preds:
            continue

        preds = invert_affine(framed_metas, preds)[0]
        
        md = dataset.df.loc[idx]
        scores = preds['scores']
        class_ids = preds['class_ids']
        rois = preds['rois']
        md['transform'] = ast.literal_eval(md['transform'].replace("'", '"'))
        if rois.shape[0] > 0:

            bbox_score = scores

            for roi_id in range(rois.shape[0]):
                score = float(bbox_score[roi_id])
                label = int(class_ids[roi_id])
                box = rois[roi_id, :]
                image_result = {
                    'image_id': idx,
                    'category_id': label + 1,
                    'score': float(score),
                    'bbox': box.tolist(),
                }
                json_results.append(image_result)
                xs, ys = geo_coords(box, md['x_offset'], md['y_offset'], md, center=False)
                results.append((get_list_polygon(xs, ys), score, label))
    
    filepath = f'coordinate_results/{set_name}_bbox_results.json'
    if os.path.exists(filepath):
        os.remove(filepath)
    json.dump(json_results, open(filepath, 'w'), indent=4)


    return results
                
def load_model(weights_path, compound_coef, device, num_classes=1, 
               ratios=[(1.0, 1.0), (1.4, 0.7), (0.7, 1.4)], scales=[2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)]):
    model = EfficientDetBackbone(compound_coef=compound_coef, num_classes=num_classes,
                                 ratios=ratios,
                                 scales=scales)
    model.load_state_dict(torch.load(weights_path, map_location=torch.device('cpu')))
    model.requires_grad_(False)
    model.eval()
    model.to(device)
    return model

## Compute model values

In [11]:
weights = 'logs/coolingTowersV2/images/efficientdet-d1_32_17000.pth'
compound_coeff = 1

model = load_model(weights, compound_coeff, device, num_classes=1)
dataset = EvaluationSet('../datasets/testcoolingTowersV2/', set_name='val',
                        compound_coeff=compound_coeff)
results = compute_coordinates(dataset, model, device, set_name='d2')

  0%|          | 0/5394 [00:00<?, ?it/s]/home/m.esguerra/.pyenv/versions/3.7.7/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 5394/5394 [05:30<00:00, 16.30it/s]


In [12]:
import pickle
with open(f'coordinate_results/coords_d1_2.pkl', 'wb') as f:
        pickle.dump(results, f)
